In [ ]:
import torch
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU memory usage
    for i in range(num_gpus):
        gpu_memory_allocated = torch.cuda.memory_allocated(i)
        gpu_memory_reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} memory allocated: {gpu_memory_allocated / (1024 ** 3):.2f} GB")
        print(f"GPU {i} memory reserved: {gpu_memory_reserved / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

In [ ]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2NativeDataType, FFF2NativeDataType, NativeDataType

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion, CellProperties, DilationedCytoMask ,\
                                    DisplaySequentialParams, IlluminationCorrection, Calculate_BIGFISH_Threshold, rescale_images,\
                                    Automatic_BIGFISH_Threshold

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

In [3]:
scope = ScopeClass() 
data = DataContainer(clear_after_error=False)
settings = Settings(name='demo')
experiment = Experiment()

In [ ]:
# you can change the parameters
experiment.initial_data_location = ['smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_0min_20220224']
                                    # 'smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_10min_20220224']

# data.local_dataset_location = [r'C:\Users\formanj\GitHub\FISH_Processing\Demos\DUSP1_Dex_10min_20220224', 
#                                  r'C:\Users\formanj\GitHub\FISH_Processing\Demos\DUSP1_Dex_0min_20220224']
experiment.FISHChannel = 0
experiment.nucChannel = 2
experiment.cytoChannel = 1
experiment.voxel_size_z = 500

settings.num_chunks_to_run = 1 # this is the limit to how many chunks you want to run and it will stop after that
# settings.psuedoCyto = 1
settings.sigma_dict = {0: 50, 1: 100, 2: 100}

scope.spot_yx = 130
scope.spot_z = 360
scope.voxel_size_yx = 100

Parameters.get_parameters()

In [ ]:
Parameters.validate()

In [ ]:
settings.load_in_mask = True
NativeDataType().run()

In [ ]:
# this will load in your data
print(data)

In [8]:
# IlluminationCorrection().run()

In [ ]:
Display().displayImage_maxProject(channel=2)

In [ ]:
settings.sigma_dict = [120, 120, 120]
IlluminationCorrection().run()

In [11]:
# if you would like to remove the original data from disk
remove_local_data_but_keep_h5().run()

In [12]:

# settings.cellpose_min_size = 500
# settings.cellpose_diameter = [180, 90]
# settings.cellpose_pretrained_model = ["GAPDH_cyto", 'DAPI_nuclei'] 

# SimpleCellposeSegmentaion().run()

In [ ]:
# settings.MAX_NUM_IMAGES_TO_AUTOMATICALLY_CALCULATE_THRESHOLD = 50
Automatic_BIGFISH_Threshold().run()

In [ ]:
settings.verbose = True
settings.bigfish_threshold = 'mean'
BIGFISH_SpotDetection().run()

In [14]:
# DilationedCytoMask().run()

In [15]:
# DisplaySequentialParams().run()

In [16]:
# CellProperties().run()

In [17]:
# inst[-1].cell_properties.columns

In [18]:
# print(inst[-1].cell_properties['nuc_label'])
# print(inst[-1].cell_properties['cell_label'])
# print(inst[-1].cell_properties['cyto_label'])


In [19]:
# Display().displayMask(channel=1, label=5)

In [20]:
# Save_Parameters().run()